In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import os

In [87]:
df = pd.read_csv("fer2013.csv")

In [88]:
# df = pd.read_csv("fer2013.csv")
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [89]:
emotions = ('angry', 'disgust', 'fear',\
                      'happy', 'sad', 'surprise', 'neutral')

In [90]:
# converting pixels in string -> int -> np array -> reshaping 48,48,1
for i in range(len(df.pixels)):
    l = df["pixels"][i].split(" ")
    l=np.array(list(map(int,l))).reshape(48,48,1)
    df["pixels"][i]=l

C:\Users\Janak\AppData\Local\Temp\ipykernel_7684\2667547461.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pixels"][i]=l


In [91]:
df["pixels"][0].shape

(48, 48, 1)

In [92]:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D,\
    Dropout,Flatten,Dense,Activation,BatchNormalization

model = Sequential()

# Convollutional Block 1
model.add(Conv2D(32,(2,2),activation='relu',input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.20))
# Convollutional Block 1
model.add(Conv2D(64,(2,2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.20))
# Convollutional Block 1
model.add(Conv2D(128,(2,2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.20))

model.add(Flatten())
# 1 Hidden Layer
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.30))
# 2 Hidden Layer
model.add(Dense(2,activation='relu'))

model.compile(
    loss="SparseCategoricalCrossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)


In [93]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 47, 47, 32)        160       
                                                                 
 batch_normalization_9 (Batc  (None, 47, 47, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 47, 47, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 47, 47, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 46, 46, 64)        8256      
                                                                 
 batch_normalization_10 (Bat  (None, 46, 46, 64)      

In [94]:
from keras.callbacks import EarlyStopping , ReduceLROnPlateau
earlystopping = EarlyStopping(patience=10)
learning_rate_reduction=ReduceLROnPlateau(monitor="val_acc",patience=2,verbose=1,factor=0.5,minn_lr=0.00001)
callbacks = [earlystopping,learning_rate_reduction]

In [95]:
train_df = df[["emotion","pixels"]][df["Usage"]=="Training"]
train_df.shape

(28709, 2)

In [165]:
train_df = df[["emotion","pixels"]][df["Usage"]=="Training"]
validate_df_x =df["pixels"][df["Usage"]=="PublicTest"]
validate_df_y = df["emotion"][df["Usage"]=="PublicTest"].astype(np.int8)
test_df = df["pixels"][df["Usage"]=="PrivateTest"]
print(train_df.shape)
print(validate_df_x.shape)
print(validate_df_y.shape)
print(test_df.shape)

(28709, 2)
(3589,)
(3589,)
(3589,)


In [166]:
x=np.array(train_df["pixels"])
y=np.array(train_df["emotion"]).astype(np.int8)
y=train_df["emotion"]

In [171]:
import torch
import numpy as np

# Assuming x is a NumPy array of pixel data

x = np.array(train_df["pixels"].apply(lambda x: np.fromstring(x, sep=' ', dtype=int)))
print(type(x))
# Convert to PyTorch tensor
x = torch.from_numpy(x)

# If needed, reshape the tensor to match the expected input shape
x = x.reshape(-1, 48, 48, 1)  # Assuming images are 48x48 and grayscale

# Convert to float and normalize if needed
x = x.float() / 255.0  # Normalize to [0, 1] range

# Now x is a PyTorch tensor that you can use in your model


<class 'numpy.ndarray'>


C:\Users\Janak\AppData\Local\Temp\ipykernel_7684\1919798020.py:6: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  x = np.array(train_df["pixels"].apply(lambda x: np.fromstring(x, sep=' ', dtype=int)))


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [148]:
import tensorflow as tf
x = (train_df["pixels"])
# y = np.array(train_df["emotion"]).astype(np.int8)

# Assuming each image is 48x48 pixels and grayscale
# x = x.reshape( -1,48, 48, 1)
print(type(y[0]))
# Convert to TensorFlow tensors
# x = tf.convert_to_tensor(x)
# y = tf.convert_to_tensor(y)

# Now you can use x and y in the model.fit() function
# model.fit(x, y, batch_size=30, epochs=30, verbose=1, validation_data=(validate_df_x, validate_df_y))



<class 'numpy.int64'>


In [149]:
model.fit(x,y,batch_size=30,epochs=30,verbose=1,validation_data=(validate_df_x,validate_df_y))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [74]:
emotions= ('angry', 'disgust', 'fear', \
                     'happy', 'sad', 'surprise', 'neutral')

In [ ]:
df_str = pd.read_csv("fer2013.csv")

In [77]:
dic  = {i :emotions[i] for i in range(len(emotions))}
df_str=df_str.replace(dic)

In [78]:

train_df_str = df_str[["emotion","pixels"]][df_str["Usage"]=="Training"]
validate_df_str = df_str["pixels"][df_str["Usage"]=="PublicTest"]
test_df_str = df_str["pixels"][df_str["Usage"]=="PrivateTest"]

In [79]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical,load_img

batch_size=15
train_datagen  = ImageDataGenerator(rotation_range = 10,
                                rescale = 1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(
    train_df_str,
    "fer2013.csv",
    x_col="pixels",
    y_col = "emotion",
    class_mode = "categorical",
    batch_size=batch_size
)

validate_datagen=ImageDataGenerator(rescale=1./255)

validate_generator = validate_datagen.flow_from_dataframe(
    validate_df_str,
    "fer2013.csv",
    x_col="pixels",
    y_col="emotion",
    target_size=(48,48,1),
    class_mode=None,
    batch_size=batch_size
)
print("validate_data generated")

test_datagen  = ImageDataGenerator(rotation_range = 10,
                                rescale = 1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
test_generator = test_datagen.flow_from_dataframe(
    test_df_str,
    "fer2013.csv",
    x_col = "pixels",
    y_col = "emotion",
    target_size=(48,48,1),
    class_mode="categorical",
    batch_size=batch_size
)

Found 0 validated image filenames belonging to 0 classes.


C:\Users\Janak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\preprocessing\image.py:989: UserWarning: Found 28709 invalid image filename(s) in x_col="pixels". These filename(s) will be ignored.
  warnings.warn('Found {} invalid image filename(s) in x_col="{}". '


KeyError: 'pixels'